In [25]:
import os

In [26]:
%pwd

'/Users/kevin/Desktop/Pokemon/Pikachu-or-Raichu--Project'

In [3]:
os.chdir("../")

In [107]:
%pwd

'/Users/kevin/Desktop/Pokemon/Pikachu-or-Raichu--Project'

In [108]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [109]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [110]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "SmallData")
        
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [111]:
import time

In [112]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [118]:
import os
import urllib.request as request
from zipfile import ZipFile
import setuptools.dist
import tensorflow as tf
import time

In [119]:
import tensorflow as tf
from pathlib import Path

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        # Load the base model from the given path
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

        # Recompile the model with a new optimizer (this ensures no optimizer issues)
        self.model.compile(optimizer=tf.keras.optimizers.Adam(), 
                           loss='categorical_crossentropy', 
                           metrics=['accuracy'])

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # Ensure this matches the model input
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Validation data generator
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Training data generator with augmentation if enabled
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        # Training data generator
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        # Save the trained model to the given path
        model.save(path)

    def train(self, callback_list: list):
        # Define steps per epoch for training and validation
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size


        # Early stopping callback: stop training when no improvement is seen in validation loss
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',  # Track validation loss to decide when to stop
            patience=3,  # Stop training if no improvement for 3 epochs
            restore_best_weights=True  # Restore model to best epoch's weights
        )

        # Add EarlyStopping to the callback list
        callback_list.append(early_stopping)

        # Train the model using the generators and callbacks
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        # Save the trained model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [120]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list  # EarlyStopping will be included here
    )

except Exception as e:
    print(f"Error occurred: {e}")
    raise e


[2024-09-20 12:38:40,462: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-20 12:38:40,465: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-20 12:38:40,466: INFO: common: Created directory at: artifacts]
[2024-09-20 12:38:40,466: INFO: common: Created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-09-20 12:38:40,467: INFO: common: Created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2024-09-20 12:38:40,470: INFO: common: Created directory at: artifacts/training]
Found 96 images belonging to 2 classes.
Found 388 images belonging to 2 classes.
Epoch 1/15


/opt/anaconda3/envs/pokemon/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 72s 5s/step - accuracy: 0.5806 - loss: 1.2438 - val_accuracy: 0.6875 - val_loss: 0.7056
Epoch 2/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.6625 - loss: 0.8535
Epoch 3/15


/opt/anaconda3/envs/pokemon/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
/opt/anaconda3/envs/pokemon/lib/python3.12/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/opt/anaconda3/envs/pokemon/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 69s 5s/step - accuracy: 0.7230 - loss: 0.6549 - val_accuracy: 0.7812 - val_loss: 0.6380
Epoch 4/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.7495 - loss: 0.6414
Epoch 5/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 66s 5s/step - accuracy: 0.8011 - loss: 0.4107 - val_accuracy: 0.8958 - val_loss: 0.2489
Epoch 6/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.7723 - loss: 0.4594
Epoch 7/15


2024-09-20 12:43:39.321318: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


15/15 ━━━━━━━━━━━━━━━━━━━━ 68s 5s/step - accuracy: 0.7985 - loss: 0.4333 - val_accuracy: 0.8958 - val_loss: 0.2361
Epoch 8/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.8347 - loss: 0.3903
Epoch 9/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.8722 - loss: 0.2632 - val_accuracy: 0.8542 - val_loss: 0.2736
Epoch 10/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.8839 - loss: 0.2900
Epoch 11/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 71s 5s/step - accuracy: 0.8960 - loss: 0.2686 - val_accuracy: 0.8958 - val_loss: 0.2907
Epoch 12/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.8574 - loss: 0.3214
Epoch 13/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 69s 5s/step - accuracy: 0.8299 - loss: 0.3482 - val_accuracy: 0.8646 - val_loss: 0.3594
